# Домашнее задание 3

## Сбор данных из интернета и генерация текстов

В этом домашнем задании вам предстоит: 
1. выбрать свой источник структурированных данных (прогноз погоды, курс акций, счет спортивных мероприятий, или что-нибудь еще) 
2. научиться извлекать данные из с выбранной веб-страницы
3. научиться генерировать текст по шаблонам

## Часть 1

Выберете источник данных, который вам симпатичен. По умолчанию, предлагаем использовать данные о погоде в Москве: 
https://www.gismeteo.ru/weather-moscow-4368/. Извлеките из html-кода страницы температуру сегодня, уровень осадков и скорость ветра.

In [1]:
import urllib
from scrapy.selector import Selector
import re
from scrapy.selector import Selector
from scrapy.http import HtmlResponse

In [2]:
url = 'https://www.gismeteo.ru/weather-moscow-4368/'
req = urllib.request.Request(url, headers={'User-Agent': "Magic Browser"})
reg_html = re.compile('<[^>]+>')
html = urllib.request.urlopen(url)
html = html.read()

In [3]:
html = urllib.request.urlopen(url)
html = html.read()

In [4]:
#общий путь
xpath = '/html/body/section/div/div/div[1]/div[3]/'
#ветер
wind_xpath = xpath + 'div[1]/div[2]/div/div[1]/div/div[5]/'
#температура
temp_xpath = xpath + 'div[1]/div[1]/div/div/div[1]/div[3]/div/div/div/'
#уровень осадков
precipitation_xpath  = xpath + 'div[1]/div[2]/div/div[1]/div/div[7]/'

In [5]:
#Посчитаем сумму осадков сегодня
def get_precipitation(html=html, reg_html=reg_html, date='today'):
    precipitation_list = []
    precipitation = 0
    
    if date == 'tomorrow':
        url = 'https://www.gismeteo.ru/weather-moscow-4368/tomorrow/'
        html = urllib.request.urlopen(url)
        html = html.read()
        
    for k in range(1, 9):
        precipitation_text = Selector(text=html).xpath(precipitation_xpath + 'div' + '[' + str(k) + ']' + '/div').extract()
        try:
            precipitation_text = re.sub(reg_html, '', precipitation_text[0]).strip()
            precipitation_text = re.sub(',', '.', precipitation_text)
            precipitation += float(precipitation_text)
        except:
            pass
        
        precipitation_list.append(precipitation_text)
    return round(precipitation, 2), precipitation_list

In [6]:
#Посчитаем сумму осадков сегодня
def get_wind(html=html, reg_html=reg_html, date='today'):
    
    wind_list = []
    wind = 0
    
    if date == 'tomorrow':
        url = 'https://www.gismeteo.ru/weather-moscow-4368/tomorrow/'
        html = urllib.request.urlopen(url)
        html = html.read()
        
    for k in range(1, 9):
        wind_text = Selector(text=html).xpath(wind_xpath + 'div' + '[' + str(k) + ']' + '/div').extract()
        try:
            wind_text = re.sub(reg_html, '', wind_text[0]).strip()
        except:
            pass
        wind_list.append(wind_text)
        
    summ = 0
    for wind in wind_list:
        list_wind = wind.split('-')
        if len(list_wind) == 1:
            avg_day = int(list_wind[0])
        else:
            avg_day = (int(list_wind[0]) + int(list_wind[1])) / 2
        summ += avg_day
    avg_wind = int(summ / 8) 
    
    return avg_wind, wind_list

In [7]:
def get_temperature(html=html, reg_html=reg_html, date='today'):
    
    if date == 'tomorrow':
        url = 'https://www.gismeteo.ru/weather-moscow-4368/tomorrow/'
        html = urllib.request.urlopen(url)
        html = html.read()
    
    
    temp_text = Selector(text=html).xpath(temp_xpath + 'div' + '[' + str(2) + ']').extract()
    temp_text = re.sub(reg_html, '', temp_text[0]).strip()
    if temp_text[0] == '+':
        return int(temp_text[1])
    else:
        return int('-' + temp_text[1])
#     temp_list = []
#     for k in range(1, 3):
#         temp_text = Selector(text=html).xpath(temp_xpath + 'div' + '[' + str(k) + ']').extract()
#         try:
#             temp_text = re.sub(reg_html, '', temp_text[0]).strip()
#         except:
#             pass
#         temp_list.append(temp_text)
        
    return temp_text

## Часть 2
Запрограммируйте шаблон, описывающие температуру, уровень осадков и скорость ветра сегодня. Например,
"Температура сегодня составляет X градусов", где Х – излеченное выше число. Сгенерируйте таким образом описание погоды сегодня. Учтите, величина X влияет на грамматическое число, использованное в единицах измерения ("двадцать градусов", но "двадцать один градус"). Используйте бибилиотеку utg или дополнительные функции pymorphy2

In [8]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

def generate_forms(word):
    p = morph.parse(word)[0]
    res = []
    lexems = p.lexeme
    if 'NOUN' in p.tag:
        for i in range(len(lexems)):
            res.append(lexems[i][4][0][1])
        #print(res)
        if 'femn' in p.tag:
            res1 = res[:5]
            res1.append(res[6])
            res1.extend(res[7:])
            res1.extend(res1[6:])
            return(res1)
        else:
            res.extend(res[6:])
            return(res)
    if 'VERB' in p.tag or 'INFN' in p.tag:
        for i in [0,1,3,2,4]:
            res.append(lexems[i][4][0][1])
        return(res)
    if 'ADJF' in p.tag:
        for i in range(18):
            res.append(lexems[i][4][0][1])
        return(res)

In [9]:
from utg import relations as r
from utg import logic
from utg import data
from utg import dictionary
from utg import words
from utg import templates
from utg import constructors

In [10]:
def get_template_temp(temp):
    
    gradus_forms = generate_forms('градус')
    # описываем существительное для словаря
    word = words.Word(type=r.WORD_TYPE.NOUN,
                            forms=gradus_forms, 
                            properties=words.Properties(r.ANIMALITY.INANIMATE, r.GENDER.MASCULINE)) 

    # создаём словарь для использования в шаблонах
    test_dictionary = dictionary.Dictionary(words=[word])

    # создаём шаблон
    template = templates.Template()

    if temp > 0:
    # externals — внешние переменные, не обязаны быть в словаре
        template.parse('В Москве сегодня [gradus] [градус|gradus] выше ноля.', externals=('gradus'))
    elif temp == 0:
        template.parse('В Москве сегодня [gradus] [градус|gradus].', externals=('gradus'))
    else:
        temp = abs(temp)
        template.parse('В Москве сегодня [gradus] [градус|gradus] ниже ноля.', externals=('gradus'))
    # осуществляем подстановку
    result = template.substitute(externals={'gradus': constructors.construct_integer(temp)},
                                 dictionary=test_dictionary)

    return result

In [11]:
def get_template_wind(avg_wind):
    
    wind_forms = generate_forms('метр')[:18]
    # описываем существительное для словаря
    word = words.Word(type=r.WORD_TYPE.NOUN,
                            forms=wind_forms, 
                            properties=words.Properties(r.ANIMALITY.INANIMATE, r.GENDER.MASCULINE)) 

    # создаём словарь для использования в шаблонах
    test_dictionary = dictionary.Dictionary(words=[word])

    # создаём шаблон
    template = templates.Template()

    if avg_wind < 7:
    # externals — внешние переменные, не обязаны быть в словаре
        template.parse('Ожидается слабый ветер, средняя скорость [avg_wind] [метр|avg_wind] в секунду.', externals=('avg_wind'))
    elif avg_wind == 0:
        template.parse('Погода безветренная.', externals=('avg_wind'))
    else:
        template.parse('Ожидается сильный ветер, средняя скорость [avg_wind] [метр|avg_wind] в секунду.', externals=('avg_wind'))
    # осуществляем подстановку
    result = template.substitute(externals={'avg_wind': constructors.construct_integer(avg_wind)},
                                 dictionary=test_dictionary)

    return result

In [12]:
def get_template_precipitation(precipitation):
    if precipitation == 0:
        return 'Осадков не ожидается.'
    elif precipitation < 2.0:
        return 'Ожидается небольшое количество осадков - ' + str(precipitation) + ' мм.'
    else:
        return 'Ожидается большое количество осадков - ' + str(precipitation) + ' мм.'

In [15]:
def main():
    temp = get_temperature()
    wind = get_wind()[0]
    precipitation = get_precipitation()[0]
    temp1 = get_template_temp(temp)
    temp2 = get_template_wind(wind)
    temp3 = get_template_precipitation(precipitation)
    
    print (temp1 + " " + temp2 + " " + temp3)

In [16]:
main()

В Москве сегодня 2 градуса выше ноля. Ожидается слабый ветер, средняя скорость 1 метр в секунду. Ожидается небольшое количество осадков - 0.2 мм.


### Часть 3 (необязательная)

Соберите данные не только за сегодняшний, но и за вчерашний день, и напишите шаблоны, которые сравнивают вчерашние и сегодняшние показатели. Например, «сегодня похолодало на Х градусов по сравнению со вчера».


In [17]:
def get_template_tommorow_temp(temp_today, temp_tomorrow):
    
    gradus_forms = generate_forms('градус')
    # описываем существительное для словаря
    word = words.Word(type=r.WORD_TYPE.NOUN,
                            forms=gradus_forms, 
                            properties=words.Properties(r.ANIMALITY.INANIMATE, r.GENDER.MASCULINE)) 

    # создаём словарь для использования в шаблонах
    test_dictionary = dictionary.Dictionary(words=[word])

    # создаём шаблон
    template = templates.Template()
    
    diff_temp = temp_tomorrow - temp_today
    
    if diff_temp > 0:
    # externals — внешние переменные, не обязаны быть в словаре
        template.parse('В Москве завтра потеплеет на [gradus] [градус|gradus].', externals=('gradus'))
    elif diff_temp == 0:
        template.parse('В Москве завтра сохранится такая же температура.', externals=('gradus'))
    else:
        template.parse('В Москве завтра похолодает на [gradus] [градус|gradus].', externals=('gradus'))
    # осуществляем подстановку
    result = template.substitute(externals={'gradus': constructors.construct_integer(diff_temp)},
                                 dictionary=test_dictionary)

    return result

In [18]:
def get_template_tommorow_wind(avg_wind_today, avg_wind_tomorrow):
    
    wind_forms = generate_forms('метр')[:18]
    # описываем существительное для словаря
    word = words.Word(type=r.WORD_TYPE.NOUN,
                            forms=wind_forms, 
                            properties=words.Properties(r.ANIMALITY.INANIMATE, r.GENDER.MASCULINE)) 

    # создаём словарь для использования в шаблонах
    test_dictionary = dictionary.Dictionary(words=[word])

    diff_wind = avg_wind_tomorrow - avg_wind_today
    # создаём шаблон
    template = templates.Template()

    if diff_wind > 0:
    # externals — внешние переменные, не обязаны быть в словаре
        template.parse('Завтра ожидается увеличение скорости ветра на [avg_wind] [метр|avg_wind] в секунду.', externals=('avg_wind'))
    elif diff_wind == 0:
        template.parse('Завтра скорость ветра сохранится на прежнем уровне.', externals=('avg_wind'))
    else:
        template.parse('Завтра ожидается уменьшение скорости ветра на [avg_wind] [метр|avg_wind] в секунду.', externals=('avg_wind'))
        diff_wind = abs(diff_wind)
    # осуществляем подстановку
    result = template.substitute(externals={'avg_wind': constructors.construct_integer(diff_wind)},
                                 dictionary=test_dictionary)

    return result

In [19]:
def get_template_tomorrow_precipitation(precipitation_today, precipitation_tomorrow):
    
    diff_precipitation = round(precipitation_tomorrow - precipitation_today, 2)
    
    if diff_precipitation == 0:
        return 'Количество осадков завтра не увеличится.'
    elif diff_precipitation > 0:
        return 'Количество осадков завтра увеличится на ' + str(diff_precipitation) + ' мм.'
    else:
        return 'Количество осадков завтра уменьшится на ' + str(diff_precipitation) + ' мм.'

In [20]:
def main_2():
    temp = get_temperature()
    wind = get_wind()[0]
    precipitation = get_precipitation()[0]
    
    temp_tomorrow = get_temperature(date='tomorrow')
    wind_tomorrow = get_wind(date='tomorrow')[0]
    precipitation_tomorrow = get_precipitation(date='tomorrow')[0]
    
    temp1 = get_template_tommorow_temp(temp, temp_tomorrow)
    temp2 = get_template_tommorow_wind(wind, wind_tomorrow)
    temp3 = get_template_tomorrow_precipitation(precipitation, precipitation_tomorrow)
    
    print (temp1 + " " + temp2 + " " + temp3)    

In [21]:
main_2()

В Москве завтра потеплеет на 2 градуса. Завтра ожидается увеличение скорости ветра на 6 м в секунду. Количество осадков завтра увеличится на 0.5 мм.
